# Querying

Let's continue with our analysis of California wildfires. In the previous section we learned how to answer questions like "what were the largest fires?", but very often we're more interested answering questions like "what were the largest fires *since the year 2019*?"

In [ ]:
import babypandas as bpd
#! We should remove the nans from the data before publishing it.
import pandas as pd
mask = pd.read_csv("../../data/calfire-full.csv").name.isnull()
fires = bpd.read_csv("../../data/calfire-full.csv")[~mask].set_index('name')

Since we already know how to sort rows by the largest fire, the next natural question to ask ourselves is how to select only the fires where the year was 2019 or later. Here's how:

In [ ]:
fires.loc[fires.get('year') >= 2019]

So what's going on in the expression above?

## Boolean Arrays

Essentially, we can select a subset of rows by using square brackets and using a {dterm}`comparison operator` on one of our columns. Recall from our introduction to {dterm}`Booleans` that we use comparison operators to return whether a comparison between two values is True or False.  In the expression above, we're getting the column of years and seeing if its 2019 or greater.

This starts to give us an idea of how the expression works beneath the surface. In fact, what happens if we just look at the result of the expression within the square brackets?

In [ ]:
fires.get('year') >= 2019

The result is a {dterm}`Boolean array` -- a sequence of True and False values.

**Note**

The term Boolean array isn't confined to just arrays. We often refer to any sequence of True and False as a Boolean array, including Series and lists.

Just like we saw that arrays and Series support element-wise operations with arithmetic, they also support element-wise *comparisons*!

When we use square brackets on a table, Babypandas expects to recieve a Boolean array that has the same length as the number of rows in the table. If the Boolean array is True in the first position, then the first row of the table will be included in the result. If the Boolean array is False in some position, then the row of the table in that same position won't be included in the result.

**Tip**

Using square brackets on a table can be read aloud as "*where*".

So the expression
```python
fires[fires.get('year') >= 2019]
```
would be read aloud as **"fires *where* the year column of fires is greater than or equal to 2019"**.

As long as the Boolean array has the same length as the number of rows in the table, we can use it. Though it's rather impractical, we could have manually created a list of True and False to pass in to our row selection.

Take the following table of five common sorting algorithms `algos` as an example:

In [ ]:
algos = bpd.DataFrame().assign(
    Algorithm=['Insertion sort', 'Merge sort', 'Quick sort', 'Bubble sort', 'Heap sort'],
    Efficiency=['O(n^2)', 'O(n log n)', 'O(n^2)', 'O(n^2)', 'O(n log n)']
).set_index('Algorithm')
algos

If we wanted to select only the rows that have an efficiency of $O(n\log n)$, we *could* manually create a Boolean array and select using that.

```{margin}
What is this mysterious notation with $O$ and $n$? It's related to the *efficiency* of an algorithm.

No need to know what they are now -- you'll cover it in DSC 40A!
```

In [ ]:
bool_arr = [False, True, False, False, True]
algos.loc[bool_arr]

Though it would almost always make quite a bit more sense to calculate the Boolean array programatically.

In [ ]:
bool_arr_calculated = algos.get('Efficiency') == 'O(n log n)'
algos.loc[bool_arr_calculated]

## Multiple conditions

In a broader sense, we're often interested in answering questions on a subset of data that satisfies certain conditions. In the wildfire example above we were interested in looking at only the fires that took place since 2019, but we might be even more interested in looking that fires that took place since 2019 *and* happened in San Diego County.

Since selecting rows using square brackets returns another DataFrame, we can store the intermediate result or rely on {dterm}`method chaining` to perform multiple selections.

In [ ]:
since_2019 = fires.loc[fires.get('year') >= 2019]
sandiego_since_2019 = since_2019.loc[since_2019.get('county') == 'San Diego County']
sandiego_since_2019

But unless we're planning on using the intermediate results stored in `since_2019`, we can use a faster approach.

Since we want to select only the rows where the year is at least 2019 *and* where the county is San Diego, we can create a Boolean array that combines these two conditions -- namely by using element-wise {dterm}`binary operators`, such as `&`, between two Boolean arrays.

```{margin}
Python is all about readability and &nbsp;a&nbsp;e&nbsp;s&nbsp;t&nbsp;h&nbsp;e&nbsp;t&nbsp;i&nbsp;c.

If we're inside of parentheses or brackets, we should freel free (and be encouraged) to break up long lines of code to make them easier to read and understand.
```

In [ ]:
fires.loc[
    (fires.get('year') >= 2019)
    & (fires.get('county') == 'San Diego County')
]

Inside the square brackets, an element-wise operation is evaluated using the `&` (we call it 'binary and') operator, which takes two Boolean arrays of the same size and returns a new Boolean array that is True only when the input arrays are *both* True.

**Tip**

Always remember your parentheses! Otherwise your Boolean arrays won't be created and the operator will break everything.

In [ ]:
fires.loc[fires.get('year') >= 2019 & fires.get('county') == 'San Diego County']

**Jupyter Tip**

If you forgot parentheses and want to add them quickly, you can select the section of code you want to surround in parentheses and then type {kbd}`(`. Jupyter will wrap your entire selection in a single pair of parentheses.

We could also use the `|` ('binary or') operator, which returns True if *either* Boolean array is True. This could be useful if we wanted to look at fires which happened in San Diego or Yolo county.

In [ ]:
fires.loc[
    (fires.get('county') == "San Diego County")
    | (fires.get('county') == "Yolo County")
]

Notice that any time that we want to use multiple conditions -- *even if they are on the same column* -- we must use this approach of either multiple selections or combining Boolean arrays.

**Question**:
 How would you use `&` to select all 'Class E' fires -- fires which burned at least 300 acres (inclusive) but less than 1000 acres?

<details><summary><b>Answer</b>:</summary>```python
fires.loc[(fires.get('acres') >= 300) & (fires.get('acres') < 300)]
```</details>

## Substring selection

Babypandas has a few extra tricks up its sleeve that makes it easier to select rows based on string conditions.

For example, it looks like there have been multiple 'Border' fires throughout history -- all with different numbers tacked on to the end of their names, like 'Border 1', 'Border 8'. Just how many 'Border' fires have their been?

We can't easily use any of the conditions we know about, since we don't know how high that number at the end might go!

To remedy this, we can use the `str.contains` on a column (or index) of our table.

In [ ]:
fires.loc[fires.index.str.contains('BORDER')]

This data is pretty clean already, but sometimes data that is entered by humans will have all sorts of inconsistencies in capitalization, punctuation, and other aspects of text. Fortunately, the string methods we covered earlier are also usable on Series of text!

A common practice is to make sure all of your text data for a given column is in the same capitalization before selecting on substrings. We can chain together the `.str.lower` method with the `.str.contains` method to quickly achieve this.

In [ ]:
fires.loc[fires.index.str.lower().str.contains('border')]

It's worth noting that if you try using the `.str` methods on a Series that doesn't contain text, you'll encounter a helpful error (once you scroll to the bottom).

In [ ]:
fires.get('year').str.contains('2019')

**Tip**

Often times, when reading an error message it's most helpful to look at the two ends of the message -- and don't get too worried about the middle bits.

The very top line points to *where* the error occurred, and the very bottom lines explain *why* the error occurred.